In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm import tqdm

In [29]:
#from sklearn.metrics import f1_score
from evaluation.evaluate_f1_partial import main as f1_score

# Data loading

In [30]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [31]:
def read_dataset(filename, splitter="\t"):
    data = []
    sentence = []
    tags = []
    with open(filename) as f:
        for line in f:
            if not line.isspace():
                word, tag = line.split(splitter)
                sentence.append(word)
                tags.append(tag.strip())
            else:
                data.append((sentence, tags))
                sentence = []
                tags = []
    return data

In [ ]:
training_data = read_dataset("data/train.tsv")
#training_data

In [33]:
test_data = read_dataset("data/test.tsv")

In [34]:
word_to_ix = {}

# For each words-list (sentence) and tags-list in each tuple of training_data
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:  # word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)  # Assign each word with a unique index
            
for sent, tags in test_data:
    for word in sent:
        if word not in word_to_ix:  # word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)  # Assign each word with a unique index


tag_to_ix = {
    "O": 0,
    "B-Object": 1,
    "I-Object": 2,
    "B-Aspect": 3,
    "I-Aspect": 4,
    "B-Predicate": 5,
    "I-Predicate": 6
}  # Assign each tag with a unique index

idx_to_tag = dict(map(reversed, tag_to_ix.items()))

EMBEDDING_DIM = 32
HIDDEN_DIM = 64

# Tagger

In [35]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

# Training

In [36]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [37]:
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

for epoch in tqdm(range(50)):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

100%|██████████| 50/50 [04:51<00:00,  5.83s/it]


# Inference

In [38]:
#Вывод слова, его класса и предсказания класса
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    tags = [idx_to_tag[int(i)] for i in tag_scores.argmax(dim=-1)]

    for x, y_true, y_pred in zip(training_data[0][0], training_data[0][1], tags):
        print(x, y_true, y_pred)

also O O
, O O
i O O
have O O
recently O O
discovered O O
advil B-Object B-Object
liquigels O O
work O O
much O O
better B-Predicate B-Predicate
and O O
faster B-Predicate B-Predicate
for O O
a O O
headache B-Aspect B-Aspect
than O O
regular O O
ibuprofen B-Object B-Object
. O O


In [39]:
#Записываем слово, его класс и предсказание класса в файл
with open("test_pred.tsv", "w") as w:
    with torch.no_grad():
        for sentence in tqdm(test_data):
            inputs = prepare_sequence(sentence[0], word_to_ix)
            tag_scores = model(inputs)
            tags = [idx_to_tag[int(i)] for i in tag_scores.argmax(dim=-1)]
            
            for i, y in zip(sentence[0], tags):
                w.write(f"{i}\t{y}\n")
            w.write("\n")

100%|██████████| 283/283 [00:00<00:00, 875.39it/s]


In [45]:
#Узнаём f1 метрику на тестовых данных
f1_score('data/test.tsv', "test_pred.tsv","out_test_f1_10steps.tsv")

In [47]:
f1_result = read_dataset("out_test_f1_10steps.tsv")
print(f1_result)

[]


Результат строгой метрики:
f1_average_strict:	0.422674
f1_aspect_strict:	0.248996
f1_object_strict:	0.305177
f1_predicate_strict:	0.776042

Результат расслабленной метрики
f1_average:	0.450414
f1_aspect:	0.277108
f1_object:	0.336058
f1_predicate:	0.797179

Описание результата:
Хороший результат показывет предсказание предикатов, когда предсказание остальных сущностей показывает низкую производительность

Мы будем учитывать баллы для всех отдельных классов, кроме тега O. Для объектов, состоящих из нескольких слов, мы используем “расслабленную” метрику: если границы прогнозируемого объекта совпадают с границами эталонного объекта, мы добавляем 1 к количеству TP (количество истинных положительных примеров). Если имеется только частичное совпадение, мы добавляем число от 0 до 1, вычисляемое как длина пересечения, разделенная на полную длину объекта.

"Расслабленная" метрика как и ожидалось показывает лучшую производительность по сравнению со строгой метрикой